In [ ]:
import numpy as np
import open3d as o3d
import pprint as pp
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import cluster, datasets, mixture
import time

In [ ]:
MODEL_INPUT_SIZE = 2048

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    #outが外れ値
    print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    return inlier_cloud

def voxcel_down(pcd,v_size):
    print("Downsample the point cloud with a voxel of %s"%v_size)
    downpcd = pcd.voxel_down_sample(voxel_size=v_size)
    return downpcd

def remove_wall(downpcd, t):
    #壁の削除
    points = np.asarray(downpcd.points)
    # 平面モデルを定義
    plano1 = pyrsc.Plane()
    # RANSACによる平面推定。平面の式とインライア(inlier)を計算。しきい値は0.01
    best_eq, best_inliers = plano1.fit(points, t)
    # 元のデータにおけるインライアの点の色を変更
    plane = downpcd.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])
    # 平面以外の点を抽出
    not_plane = downpcd.select_by_index(best_inliers, invert=True)
    return not_plane, plane

def remove_desk(not_plane,t):
    #机の削除
    desk_points = np.asarray(not_plane.points)
    # 平面モデルを定義
    plano1 = pyrsc.Plane()
    # RANSACによる平面推定。平面の式とインライア(inlier)を計算。しきい値は0.01
    # 机の部分を削除（多分閾値でいけるはず）
    best_eq, desk_inliers = plano1.fit(desk_points, t)
    # 元のデータにおけるインライアの点の色を変更（赤色に変更）
    desk_place = not_plane.select_by_index(desk_inliers).paint_uniform_color([1, 0, 0])
    # 平面以外の点を抽出
    desk_not_plane = not_plane.select_by_index(desk_inliers, invert=True)
    return desk_not_plane, desk_place

def farthest_point_sample(point, npoint):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    point = point[centroids.astype(np.int32)]
    return point

def l2_norm(a, b):
    return ((a - b) ** 2).sum(axis=1)

def farthest_point_sampling(pcd, k, metrics=l2_norm):
    indices = np.zeros(k, dtype=np.int32)
    points = np.asarray(pcd.points)
    distances = np.zeros((k, points.shape[0]), dtype=np.float32)
    indices[0] = np.random.randint(len(points))
    farthest_point = points[indices[0]]
    min_distances = metrics(farthest_point, points)
    distances[0, :] = min_distances
    
    for i in range(1, k):
        indices[i] = np.argmax(min_distances)
        farthest_point = points[indices[i]]
        distances[i, :] = metrics(farthest_point, points)
        min_distances = np.minimum(min_distances, distances[i, :])
    pcd = pcd.select_by_index(indices)
    
    return pcd

In [ ]:
#ファイル名指定（plyファイル）
pcd = o3d.io.read_point_cloud("frame.1-4.s303.df1.5.ply")
#voxel_sizeを指定
v_size = 0.01
#voxcel_downsampling
downpcd = voxcel_down(pcd, v_size)

#o3d.visualization.draw_geometries([downpcd,])

In [ ]:
#閾値設定
threshold = 0.01
#remove wall
not_plane, plane = remove_wall(downpcd, threshold)
pp.pprint(not_plane)
#3次元座標上の値を出力
# pp.pprint(np.asarray(not_plane.points))
#可視化
#o3d.visualization.draw_geometries([not_plane, plane])

In [ ]:
#外れ値除去をやる
#statistical
print("Radius oulier removal")
cl, ind = not_plane.remove_statistical_outlier(nb_neighbors=30,std_ratio=2.0)
processed_point = display_inlier_outlier(not_plane, ind)

In [ ]:
#閾値設定
desk_threshold = 0.01
#机の削除
desk_not_plane, desk_plane = remove_desk(processed_point, desk_threshold)
pp.pprint(desk_not_plane)
#o3d.visualization.draw_geometries([desk_not_plane, desk_plane])
#raise SystemExit

In [ ]:
#外れ値除去をやる
#radisu
print("Radius oulier removal")
cl, ind = desk_not_plane.remove_radius_outlier(nb_points=20, radius=0.05)
processed_point = display_inlier_outlier(desk_not_plane, ind)
pp.pprint(processed_point)
#o3d.visualization.draw_geometries([processed_point])
#raise SystemExit

In [ ]:
# Cropping
input_pcd = processed_point
print("\nOriginal bounds")
pp.pprint(input_pcd.get_min_bound())
pp.pprint(input_pcd.get_max_bound())

boundingbox = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=(-0.53,  -0.50,   -1.49),
    max_bound=( 0.53,   0.58,  -0.8))
cropped_pcd = input_pcd.crop(boundingbox)

print("\nNew bounds")
pp.pprint(cropped_pcd.get_min_bound())
pp.pprint(cropped_pcd.get_max_bound())

processed_point = cropped_pcd
pp.pprint(processed_point)
#o3d.visualization.draw_geometries([processed_point])
#raise SystemExit

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels_o3d = processed_point.cluster_dbscan(eps=0.05, min_points=3, print_progress=True)
    labels = np.array(labels_o3d)

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
processed_point.colors = o3d.utility.Vector3dVector(colors[:, :3])

cluster_max_point = np.array(processed_point.points)
cluster_point = []

point_clusters = dict()
point_counters = dict()
for i,j in enumerate(labels):
    try:
        point_clusters[j].append(cluster_max_point[i])
        point_counters[j] += 1
    except KeyError:
        point_clusters[j] = [cluster_max_point[i],]
        point_counters[j] = 1
        
for key, num_points in point_counters.items():
    print(f"Cluster #{key} has {num_points} points")
    
sorted_clusters = [k for k, v in sorted(
                        point_counters.items(), 
                        key=lambda item: item[1], 
                        reverse=True)]

biggest_cluster = sorted_clusters[0]
cluster_point = point_clusters[biggest_cluster]
print(f"\nBiggest cluster #{biggest_cluster} has {len(cluster_point)} points")
        
main_pcd = o3d.geometry.PointCloud()
main_pcd.points = o3d.utility.Vector3dVector(cluster_point)
#o3d.visualization.draw_geometries([main_pcd])

print("Downscaling to model input size")
fps = farthest_point_sampling(main_pcd, MODEL_INPUT_SIZE)

#numpy→open3d型に変更
main_pcd = o3d.geometry.PointCloud()
main_pcd.points = o3d.utility.Vector3dVector(fps.points)

pp.pprint(main_pcd)

In [ ]:
# 可視化
o3d.visualization.draw_geometries([main_pcd])